## MMinte multiprocessor

mminte-mp is a multiprocessor implementation of the core MMinte algorithm that improves performance when analyzing a large microbial network. There are two functions for calculating the pairwise interactions in the microbial network.

1. `create_interaction_models()` takes a list of single species models, creates all of the two species models needed for the analysis, and returns a list of two species models.
2. `calculate_growth_rates()` takes a list of two species models and a media file, calculates the growth rates of the pairs in the specified media, and returns a DataFrame with the results.

For a single species model to be used by mminte-mp it must meet these requirements:

1. Reaction and metabolite IDs must have a compartment suffix using one of two
   types. A "bigg" compartment suffix has the format "[x]" where x is a single
   character compartment ID (for example, "[c]" for cytosol compartment). A
   "modelseed" compartment suffix has the format "\_x" where x is a single character
   compartment ID (for example "\_c" for cytosol compartment). You can mix ID types
   in the same model.
2. There can be only one objective to optimize for growth in each source model.
3. Exchange reactions are identified by an 'EX\_' prefix on the reaction ID.
4. Exchange reactions have only one metabolite with a negative coefficient.

A media file is in JSON format and contains a dictionary with an exchange reaction ID as the key and lower and upper bounds of the exchange reaction as the value. For example,

    {'EX_h2o': [0.0, 1000.0],
     'EX_h2s': [-1.0, 1000.0],
     'EX_pi': [-10.0, 1000.0],
     ...
    }


In [1]:
import mminte

First, create the two species community models for all pairs in a microbial network. In the example below there are four species in the microbial network which creates six two species models.

In [2]:
pair_models = mminte.create_interaction_models(['../mminte/test/data/BT.sbml', '../mminte/test/data/FP.sbml', '../mminte/test/data/HP.sbml', '../mminte/test/data/KP.sbml'], output_folder='../mminte/test/data')

Second, calculate growth rates for all pairs in the microbial network in the specified media condition. The media conditions are specified in a json file that is a dictionary keyed by exchange reaction ID and value is a tuple with lower bound and upper bound.

In [3]:
growth_rates = mminte.calculate_growth_rates(pair_models, '../mminte/test/data/western.json')

The results are returned in a DataFrame with the following columns:

1. A_ID - ID of first species
2. B_ID - ID of second species
3. TYPE - Type of interaction between the two species
4. TOGETHER - Growth rate of both species together
5. A_TOGETHER - Growth rate of first species when grown in presence of second species
6. B_TOGETHER - Growth rate of second species when grown in presence of first species
7. A_ALONE - Growth rate of first species when grown in absense of second species
8. B_ALONE - Growth rate of second species when grown in absense of first species
9. A_CHANGE - Percent change in growth rate of first species
10. B_CHANGE - Percent change in growth rate of second species

In [4]:
growth_rates

,A_ID,B_ID,TYPE,TOGETHER,A_TOGETHER,B_TOGETHER,A_ALONE,B_ALONE,A_CHANGE,B_CHANGE
0,BT,FP,Parasitism,0.495075,2.774626e-01,0.217612,0.440738,0.169338,-0.370460,0.285078
1,BT,HP,Parasitism,0.500853,3.645653e-02,0.464396,0.440738,0.197557,-0.917283,1.350692
2,BT,KP,Parasitism,0.586633,0.000000e+00,0.586633,0.440738,0.510884,-1.000000,0.148269
3,FP,HP,Commensalism,0.431667,1.775634e-01,0.254104,0.169338,0.197557,0.048574,0.286228
4,FP,KP,Amensalism,0.545572,3.556715e-14,0.545572,0.169338,0.510884,-1.000000,0.067898
5,HP,KP,Amensalism,0.540178,0.000000e+00,0.540178,0.197557,0.510884,-1.000000,0.057339
